# DS 203 Final Project

## Team members
1. 210040098 Mohak Vyas 
2. 210040125 Rohan Choudhary
3. 210040131 Sahil Barbade
4. 210040054 Harendra Pratap Singh Baghel

## EDA

In [6]:
import cv2
import numpy as np
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm


## FEATURE-EXTRACTION - METHOD-1
def find_blob_properties(image_path):
    # Read the image
    image = cv2.imread(image_path)
    blob_properties = []

    # applying some filters before drawing contours
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_image, (3, 3), .8)
    edges = cv2.Canny(blurred, 8, 50)
    
    
    _, binary_image = cv2.threshold(edges, 200, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        epsilon = 0.002* cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) < 3:
            continue

        sides = len(approx)
        
        contour_area = cv2.contourArea(contour)
        
        x, y, w, h = cv2.boundingRect(approx)
        bounding_box_area = w * h

        # Drawing the image with contour
        cv2.drawContours(image, [approx], 0, (155, 155, 0), 2)
        cv2.putText(image, str(sides), tuple(approx[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(image, (x, y), (x + w, y+h), (0, 255, 0), 2)

        # cv2.startWindowThread()
        # cv2.imshow('Polygons with Sides Count', image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
        blob_properties.append({ "num_vertices": sides, "bounding_box": (x, y, w, h), "contour_area": contour_area, "bounding_box_area": bounding_box_area})
    
    return image, blob_properties
        
    
# def print_image(image_detail):  # TODO: add contour onto the images
#                                 # TILL THEN USE THE PRINT IN FUNC find_blob_properties(image_path)
#     folder_path = "E7-images"
#     for key, value in image_detail.items():
#         formatted_index = str(key).zfill(4)
#         image_path = os.path.join(folder_path, f"{formatted_index}.jpg")
#         image = cv2.imread(image_path)
#         # Draw contours, text, and bounding boxes for each blob in the image
#         for blob_properties in value:
#           x, y, w, h = blob_properties["bounding_box"]

#           cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box

#         # cv2.startWindowThread()
#         # cv2.imshow('Polygons with Sides Count', image)
#         # cv2.waitKey(0)
#         # cv2.destroyAllWindows()



def extract_VGG_features(img_path):
   # Function to extract features from an image
    model = models.vgg19(pretrained=True).features
    model.eval()

    # Define image transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    img = Image.open(img_path)
    
    # Resize image to 640x480
    img = img.resize((640, 480))
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = model(img)
    return features.flatten().numpy() 


from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

## METHOD-1 :: K-means using basic geometric features
def cluster_indices_kmeans(data, num_clusters):

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled)

    # Plotting the clusters
    plt.figure(figsize=(10, 6))
    for i in range(num_clusters):
        plt.scatter(data[cluster_labels == i, 0], data[cluster_labels == i, 1], label=f'Cluster {i+1}')

    plt.title('Clusters using K-means (Method 1)')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.grid(True)
    plt.show()


## METHOD-2 :: K-means using high quality CNN features
def cluster_indices_CNN_feature(data, num_clusters):

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42)
    cluster_labels = kmeans.fit_predict(data)

    # Plotting the clusters
    plt.figure(figsize=(10, 6))
    for i in range(num_clusters):
        plt.scatter(data[cluster_labels == i, 0], data[cluster_labels == i, 1], label=f'Cluster {i+1}')

    plt.title('Clusters using K-means (Method 2)')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.grid(True)
    plt.show()

        


## METHOD-3 :: Hierarchical using high quality CNN features


# Based on the elbow method, you can visually inspect the graph to find the optimal number of clusters.

df = None
    
def main():
    global df
    folder_path = "E7-images"
    image_files = os.listdir(folder_path)
    features_method_1 = {"num_vertices" : [], "x": [], "y": [], "w" : [], "h" :[], "c_area" : []}  # dictionary that has all details about the images
    features_method_2 = []
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        ## METHOD-1 Features
        image_with_blobs, blob_properties = find_blob_properties(image_path)   # Find blob properties and draw them on the image
        features_method_1["num_vertices"].append(blob_properties[0]["num_vertices"])
        features_method_1["x"].append(blob_properties[0]["bounding_box"][0])
        features_method_1["y"].append(blob_properties[0]["bounding_box"][1])
        features_method_1["w"].append(blob_properties[0]["bounding_box"][2])
        features_method_1["h"].append(blob_properties[0]["bounding_box"][3])
        features_method_1["c_area"].append(blob_properties[0]["contour_area"])
        
        ## METHOD-2 Features
        feature = extract_VGG_features(image_path)
        features_method_2.append(feature)
        
    df = pd.DataFrame(features_method_1)
    cluster_indices_kmeans(df)
    
    cluster_indices_CNN_feature(features_method_2)

    
if __name__ == "__main__":
    main()


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: cluster_indices_kmeans() missing 1 required positional argument: 'num_clusters'

In [4]:
df

num_vertices    x   y    w    h    c_area
0                6  102  73  452  339  134731.5
1                9  102  73  452  338  130780.5
2               20  102  73  452  338  107663.5
3               14  102  74  452  337  101004.0
4                5  101  74  454  338  146805.5
...            ...  ...  ..  ...  ...       ...
1178            21  102  74  453  336  115441.5
1179             3  102  74  452  337   76794.0
1180            19  101  74  453  337  105909.0
1181             8  101  73  454  339  147651.5
1182             7  101  74  453  338   94400.5

[1183 rows x 6 columns]

In [5]:
df.to_csv('your_file_name.csv', index=False)